# Carga de información a la base de datos BBDD_Hoteles

En este documento realizamos la carga de los datos limpiados a la base de sql.

In [1]:
import pandas as pd
import numpy as np
import psycopg2 as ps

In [4]:
# Importamos el csv con la información de las reservas, los hoteles y los clientes
data = pd.read_csv("../data/reservas_hoteles_limpio.csv", parse_dates=['fecha_reserva', 'inicio_estancia', 'final_estancia'])
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id_reserva       15000 non-null  object        
 1   id_cliente       15000 non-null  int64         
 2   nombre           15000 non-null  object        
 3   apellido         15000 non-null  object        
 4   mail             15000 non-null  object        
 5   competencia      15000 non-null  bool          
 6   fecha_reserva    9828 non-null   datetime64[ns]
 7   inicio_estancia  15000 non-null  datetime64[ns]
 8   final_estancia   15000 non-null  datetime64[ns]
 9   id_hotel         15000 non-null  int64         
 10  precio_noche     9776 non-null   float64       
 11  nombre_hotel     9828 non-null   object        
 12  estrellas        9828 non-null   float64       
 13  ciudad           9828 non-null   object        
dtypes: bool(1), datetime64[ns](3), float64

In [5]:
# Importamos el csv con los eventos de Madrid sacados de la API
eventos = pd.read_csv("../data/eventos_madrid.csv", parse_dates=['fecha_inicio', 'fecha_fin'])
eventos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217 entries, 0 to 216
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   nombre_evento  217 non-null    object        
 1   url_evento     217 non-null    object        
 2   codigo_postal  201 non-null    float64       
 3   direccion      201 non-null    object        
 4   horario        134 non-null    object        
 5   organizacion   202 non-null    object        
 6   fecha_inicio   217 non-null    datetime64[ns]
 7   fecha_fin      217 non-null    datetime64[ns]
dtypes: datetime64[ns](2), float64(1), object(5)
memory usage: 13.7+ KB


## Inicialización de la conexión con la BBDD y el cursor

In [7]:
# Creamos la conexión a la base de datos
# Vamos a crear una conexión a la base de datos.
conn = ps.connect(
    dbname = "BBDD_Hoteles", # base a la que nos queremos conectar
    user = "postgres",
    password = "admin",
    host = "localhost",
    port = "5432" # puerto en el que s eencuentra postgres
)

In [8]:
# Creamos un cursor el cual nos va a permitir ejecutar querys.
cur = conn.cursor()

In [9]:
# COmprobamos que la conexión está creada y conectada
cur.execute("SELECT version();")
cur.fetchone() 

('PostgreSQL 16.4, compiled by Visual C++ build 1940, 64-bit',)

## Inserción de datos

In [8]:
# COmprobamos que no haya valores duplicados en ambos dataframes
print(data.duplicated().sum())
print(eventos.duplicated().sum())

0
0


### Tabla ciudad

In [ ]:
# Creamos la query de inserción de los datos de ciudad
insert_query_ciudad = """ 
                        INSERT INTO ciudad (nombre_ciudad)
                        VALUES (%s) 
"""

In [ ]:
# Ejecutamos la query y le indicamos que solo tenemos una ciudad que es Madrid
cur.execute(insert_query_ciudad, ("Madrid"))
conn.commit()

### Tabla eventos

In [ ]:
# cambiar tipo de dato de codigo postal a int
eventos.columns

Index(['nombre_evento', 'url_evento', 'codigo_postal', 'direccion', 'horario',
       'organizacion', 'fecha_inicio', 'fecha_fin'],
      dtype='object')

In [ ]:
# Sacamos el id de ciudad de la tabla de ciudad
cur.execute("SELECT ciudad, id_ciudad FROM ciudad")
ciudad = dict(cur.fetchone())
ciudad

In [ ]:
data_insert_eventos = []

for i, row in eventos.iterrows():
    nombre_evento = row["nombre_evento"]
    url_evento = row["url_evento"]
    codigo_postal = row["codigo_postal"]
    direccion = row["direccion"]
    horario = row["horario"]
    fecha_inicio = row["fecha_inicio"]
    fecha_fin = row["fecha_fin"]
    organizacion = row["organizacion"]
    id_ciudad = ciudad.get(row["id_ciudad"])

    data_insert_eventos.append(nombre_evento, url_evento, codigo_postal, direccion, horario, fecha_inicio, fecha_fin, organizacion, id_ciudad)

In [ ]:
# Creamos la query de inserción de los datos de eventos
insert_query_eventos = """ 
                        INSERT INTO eventos (nombre_evento, url_evento, codigo_postal, direccion, horario,
                        fecha_inicio, fecha_fin,organizacion, id_ciudad)
                        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
"""

In [ ]:
# Ejecutamos la query y le indicamos los valores 
cur.execute(insert_query_eventos, data_insert_eventos)
conn.commit()

### Tabla hoteles

In [ ]:
# revisar tipo de datos
data.columns

Index(['id_reserva', 'id_cliente', 'nombre', 'apellido', 'mail', 'competencia',
       'fecha_reserva', 'inicio_estancia', 'final_estancia', 'id_hotel',
       'precio_noche', 'nombre_hotel', 'estrellas', 'ciudad'],
      dtype='object')

In [ ]:
data_insert_hotels = []

for _, row in data.iterrows():
    nombre_hotel = row["nombre_hotel"]
    competencia = row["competencia"]
    estrellas = row["estrellas"]
    id_ciudad = ciudad.get(row["id_ciudad"])

    data_insert_eventos.append(nombre_hotel, competencia, estrellas, id_ciudad)

In [ ]:
# Creamos la query de inserción de los datos de ciudad
insert_query_hoteles = """ 
                        INSERT INTO eventos (nombre_hotel, competencia, estrellas, id_ciudad)
                        VALUES (%s, %s, %s, %s)
"""

In [ ]:
# Ejecutamos la query y le indicamos los valores 
cur.execute(insert_query_hoteles, data_insert_hotels)
conn.commit()

### Tabla clientes

In [ ]:
# revisar tipo de datos
data.columns

Index(['id_reserva', 'id_cliente', 'nombre', 'apellido', 'mail', 'competencia',
       'fecha_reserva', 'inicio_estancia', 'final_estancia', 'id_hotel',
       'precio_noche', 'nombre_hotel', 'estrellas', 'ciudad'],
      dtype='object')

In [ ]:
data_insert_clientes = []

for _, row in data.iterrows():
    id_cliente = row["id_cliente"]
    nombre = row["nombre"],
    apellido = row["apellido"],
    mail = row["mail"]

    data_insert_eventos.append(id_cliente, nombre, apellido, mail)

In [ ]:
# Creamos la query de inserción de los datos de ciudad
insert_query_clientes = """ 
                        INSERT INTO eventos (id_cliente, nombre, apellido, mail)
                        VALUES (%s, %s, %s, %s)
"""

In [ ]:
# Ejecutamos la query y le indicamos los valores 
cur.execute(insert_query_clientes, data_insert_clientes)
conn.commit()

### Tabla reservas

In [ ]:
# revisar tipo de datos
data.columns

Index(['id_reserva', 'id_cliente', 'nombre', 'apellido', 'mail', 'competencia',
       'fecha_reserva', 'inicio_estancia', 'final_estancia', 'id_hotel',
       'precio_noche', 'nombre_hotel', 'estrellas', 'ciudad'],
      dtype='object')

In [ ]:
# Sacamos el id de clientes de la tabla de clientes
cur.execute("SELECT cliente, id_cliente FROM clientes")
clientes = dict(cur.fetchall())
clientes

In [ ]:
# Sacamos el id de hotel de la tabla de hoteles
cur.execute("SELECT hotel, id_hotel FROM hoteles")
hoteles = dict(cur.fetchall())
hoteles

In [ ]:
data_insert_reservas = []

for _, row in data.iterrows():
    id_reserva = row["id_reserva"]
    fecha_reserva = row["fecha_reserva"],
    inicio_estancia = row["inicio_estancia"],
    final_estancia = row["final_estancia"],
    precio_noche = data["precio_noche"],
    id_cliente = clientes.get(row["id_cliente"]),
    id_hotel = hoteles.get(row["id_hotel"])

    data_insert_eventos.append(id_reserva, fecha_reserva, inicio_estancia, final_estancia, precio_noche, id_cliente, id_hotel)

In [ ]:
# Creamos la query de inserción de los datos de ciudad
insert_query_reservas = """ 
                        INSERT INTO eventos (id_reserva, fecha_reserva, inicio_estancia, final_estancia, precio_noche, id_cliente, id_hotel)
                        VALUES (%s, %s, %s, %s, %s, %s, %s)
"""

In [ ]:
# Ejecutamos la query y le indicamos los valores 
cur.execute(insert_query_reservas, data_insert_reservas)
conn.commit()

## Cierre de la conexión

In [13]:
# una vez hemos terminado de trabajar es necesario cerrar la conexión y el cursor.
cur.close()
conn.close()